In [3]:
from pyo import *

In [4]:
CHUNK = 1024  # Número de muestras por buffer
SRATE = 44100  # Frecuencia de muestreo
AMP = 0.1 # Amplitud general del sonido
DEVICE_OUT = 6  # Dispositivo de audio a utilizar (puede cambiar según el sistema)
DEVICE_IN = 6  # Dispositivo de audio a utilizar (puede cambiar según el sistema)

def start_server():
    """Inicia el servidor de audio y lo configura."""
    server = Server(audio='alsa', nchnls=1)
    server.amp = AMP
    server.setSamplingRate(SRATE)
    server.setBufferSize(CHUNK)
    server.setInputDevice(DEVICE_IN) # & = default
    server.setOutputDevice(DEVICE_OUT) # & = default
    server.setNchnls(1)  # Número de canales de salida aparentemente solo vale 1
    server.boot()
    return server

# s = start_server()

In [ ]:
s = start_server()
snd = SndTable(SNDS_PATH + "/transparent.aif")
env = HannTable()
pos = Phasor(snd.getRate()*.25, 0, snd.getSize())
dur = Noise(.001, .1)
g = Granulator(snd, env, [1, 1.001], pos, dur, 24, mul=.1).out()
s.gui(locals())

Pyo warning: Unknown audio type. Using Portaudio
Pyo warning: Portmidi warning: could not open midi input 1 (Midi Through Port-0): PortMidi: Invalid device ID
Pyo warning: Portmidi warning: could not open midi output 0 (Midi Through Port-0): PortMidi: Invalid device ID
Pyo warning: Portmidi closed.
